In [105]:
import numpy as np
import keras as K
import tensorflow as tf
import os
import pandas as pd
import re

**Data preprocessing section, combin and clean data from different sources**

In [106]:
from google.colab import drive
drive.mount('/content/gdrive')
path_prefix = "/content/gdrive/MyDrive/733_final_project/sentiment"

import sys
sys.path.insert(1, path_prefix)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [107]:
!ls "/content/gdrive/My Drive/733-final-project/sentiment"


BTC_tweets_daily_example.csv  twitter_kaggle_training.csv
Twitter+data+in+sheets.xlsx   twitter_kaggle_validation.csv


In [220]:

datasets_folder = '/content/gdrive/My Drive/733-final-project/sentiment'
BTC_tweets_daily_example_dir = datasets_folder + '/BTC_tweets_daily_example.csv'
twitter_kaggle_training_dir = datasets_folder + '/twitter_kaggle_training.csv'
twitter_kaggle_validation_dir = datasets_folder + '/twitter_kaggle_validation.csv'
Twitter_data_in_sheets_dir = datasets_folder + '/Twitter+data+in+sheets.xlsx'


BTC_tweets_daily_example = pd.read_csv(BTC_tweets_daily_example_dir)
twitter_kaggle_training = pd.read_csv(twitter_kaggle_training_dir)
twitter_kaggle_validation = pd.read_csv(twitter_kaggle_validation_dir)
Twitter_data_in_sheets = pd.read_excel(Twitter_data_in_sheets_dir)

save_df_dir = datasets_folder+ '/Cleaned_combined_df.csv'
  

In [221]:
BTC_tweets_daily_example.head()

,Unnamed: 0,Date,Tweet,Screen_name,Source,Link,Sentiment,sent_score,New_Sentiment_Score,New_Sentiment_State
0,0,Fri Mar 23 00:40:32 +0000 2018,"RT @ALXTOKEN: Paul Krugman, Nobel Luddite. I h...",myresumerocket,[],"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",['neutral'],0.0,0.000000,0.0
1,1,Fri Mar 23 00:40:34 +0000 2018,@lopp @_Kevin_Pham @psycho_sage @naval But @Pr...,BitMocro,[u'Bitcoin'],"<a href=""http://twitter.com/download/android"" ...",['neutral'],0.0,0.000000,0.0
2,2,Fri Mar 23 00:40:35 +0000 2018,RT @tippereconomy: Another use case for #block...,hojachotopur,"[u'blockchain', u'Tipper', u'TipperEconomy']","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",['positive'],1.0,0.136364,1.0
3,3,Fri Mar 23 00:40:36 +0000 2018,free coins https://t.co/DiuoePJdap,denies_distro,[],"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",['positive'],1.0,0.400000,1.0
4,4,Fri Mar 23 00:40:36 +0000 2018,RT @payvxofficial: WE are happy to announce th...,aditzgraha,[],"<a href=""http://twitter.com/download/android"" ...",['positive'],1.0,0.468182,1.0


In [222]:
selected_columns = ['Tweet', 'sent_score']
filtered_BTC_tweets = BTC_tweets_daily_example[selected_columns]
filtered_BTC_tweets = filtered_BTC_tweets.copy()
filtered_BTC_tweets.head()


,Tweet,sent_score
0,"RT @ALXTOKEN: Paul Krugman, Nobel Luddite. I h...",0.0
1,@lopp @_Kevin_Pham @psycho_sage @naval But @Pr...,0.0
2,RT @tippereconomy: Another use case for #block...,1.0
3,free coins https://t.co/DiuoePJdap,1.0
4,RT @payvxofficial: WE are happy to announce th...,1.0


In [223]:
def clean_tweet(tweet):
    if not isinstance(tweet, str):
      tweet = str(tweet)
    tweet = re.sub(r'@\w+', '', tweet)  # Remove @username mentions
    tweet = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', tweet)  # Remove URLs
    tweet = re.sub(r'#(\w+)', r'\1', tweet)  # Remove '#' from hashtags
    tweet = re.sub(r'\bRT :', '', tweet)  # Remove 'RT :' from retweets

    return tweet.strip()


In [224]:
filtered_BTC_tweets['Cleaned_Tweet'] = filtered_BTC_tweets['Tweet'].apply(clean_tweet)
filtered_BTC_tweets= filtered_BTC_tweets[['Cleaned_Tweet', 'sent_score']]

**Cleaning Done for dataset1**

In [225]:
filtered_BTC_tweets.head()

,Cleaned_Tweet,sent_score
0,"Paul Krugman, Nobel Luddite. I had to tweak th...",0.0
1,But (dum b a ss) said you know nothing about ...,0.0
2,Another use case for blockchain and Tipper. Th...,1.0
3,free coins,1.0
4,WE are happy to announce that PayVX Presale Ph...,1.0


In [226]:
filtered_BTC_tweets['Cleaned_Tweet'].dtypes

dtype('O')

In [227]:
unique_values = filtered_BTC_tweets['sent_score'].value_counts()
print(unique_values)

 1.0    22937
 0.0    21939
-1.0     5983
Name: sent_score, dtype: int64


In [228]:
twitter_kaggle_training.head()

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [229]:
unique_values_2 = twitter_kaggle_training['Positive'].value_counts()
print(unique_values_2)

Negative      22542
Positive      20831
Neutral       18318
Irrelevant    12990
Name: Positive, dtype: int64


In [230]:
# rename columns
twitter_kaggle_training = twitter_kaggle_training.rename(columns={twitter_kaggle_training.columns[-1]: 'tweet'})
twitter_kaggle_training = twitter_kaggle_training.rename(columns={twitter_kaggle_training.columns[-2]: 'sent_score'})
twitter_kaggle_training.head()

,2401,Borderlands,sent_score,tweet
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [231]:
twitter_kaggle_training['Cleaned_Tweet'] = twitter_kaggle_training['tweet'].apply(clean_tweet)

In [232]:
twitter_kaggle_training = twitter_kaggle_training[['Cleaned_Tweet', 'sent_score']]

In [233]:
# Replace the values
replacement_values = {'sent_score': {'Positive': 1.0, 'Negative': -1.0, 'Neutral': 0.0}}
twitter_kaggle_training = twitter_kaggle_training.replace(replacement_values)

**Cleaning Done For dataset2**

In [234]:
twitter_kaggle_training = twitter_kaggle_training[twitter_kaggle_training['sent_score'] != 'Irrelevant']

twitter_kaggle_training.head()

,Cleaned_Tweet,sent_score
0,I am coming to the borders and I will kill you...,1.0
1,im getting on borderlands and i will kill you ...,1.0
2,im coming on borderlands and i will murder you...,1.0
3,im getting on borderlands 2 and i will murder ...,1.0
4,im getting into borderlands and i can murder y...,1.0


In [235]:
twitter_kaggle_training['sent_score'].dtypes

dtype('O')

In [236]:
twitter_kaggle_validation.head()

,3364,Facebook,Irrelevant,"I mentioned on Facebook that I was struggling for motivation to go for a run the other day, which has been translated by Tom’s great auntie as ‘Hayley can’t get out of bed’ and told to his grandma, who now thinks I’m a lazy, terrible person 🤣"
0,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
1,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
2,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
3,4433,Google,Neutral,Now the President is slapping Americans in the...
4,6273,FIFA,Negative,Hi @EAHelp I’ve had Madeleine McCann in my cel...


In [237]:
Twitter_data_in_sheets.head()

,TweetID,Weekday,Hour,Day,Lang,IsReshare,Reach,RetweetCount,Likes,Klout,Sentiment,text,LocationID,UserID
0,tw-682712873332805633,Thursday,17.0,31.0,en,0.0,44.0,0.0,0.0,35.0,0.0,We are hiring: Senior Software Engineer - Prot...,3751.0,tw-40932430
1,tw-682713045357998080,Thursday,17.0,31.0,en,1.0,1810.0,5.0,0.0,53.0,2.0,RT @CodeMineStatus: This is true Amazon Web Se...,3989.0,tw-3179389829
2,tw-682713219375476736,Thursday,17.0,31.0,en,0.0,282.0,0.0,0.0,47.0,0.0,Devops Engineer Aws Ansible Cassandra Mysql Ub...,3741.0,tw-4624808414
3,tw-682713436967579648,Thursday,17.0,31.0,en,0.0,2087.0,4.0,0.0,53.0,0.0,Happy New Year to all those AWS instances of o...,3753.0,tw-356447127
4,tw-682714048199311366,Thursday,17.0,31.0,en,0.0,953.0,0.0,0.0,47.0,0.0,Amazon is hiring! #Sr. #International Tax Mana...,3751.0,tw-3172686669


In [238]:
unique_values4 = Twitter_data_in_sheets[' Sentiment'].value_counts()

print(unique_values4)
print('max', Twitter_data_in_sheets[' Sentiment'].max())
print('min', Twitter_data_in_sheets[' Sentiment'].min())

 0.000000    67424
 1.000000     8771
 2.000000     8587
 3.000000     3945
-1.000000     1770
             ...  
 0.186667        1
-0.140000        1
 0.356250        1
 0.875000        1
 0.136351        1
Name:  Sentiment, Length: 318, dtype: int64
max 7.33333349227905
min -6.0


In [239]:
def convert_score(score):
    if score > 0:
        return 1
    elif score < 0:
        return -1
    else:
        return 0

In [240]:
Twitter_data_in_sheets['Cleaned_Tweet'] = Twitter_data_in_sheets[' text'].apply(clean_tweet)

In [241]:
# Apply the custom function to the 'sentiment_score' column
Twitter_data_in_sheets['sent_score'] = Twitter_data_in_sheets[' Sentiment'].apply(convert_score)


In [242]:
Twitter_data_in_sheets= Twitter_data_in_sheets[['Cleaned_Tweet', 'sent_score']]

In [243]:
Twitter_data_in_sheets.head()

,Cleaned_Tweet,sent_score
0,We are hiring: Senior Software Engineer - Prot...,0
1,This is true Amazon Web Services php html htm...,1
2,Devops Engineer Aws Ansible Cassandra Mysql Ub...,0
3,Happy New Year to all those AWS instances of o...,0
4,Amazon is hiring! Sr. International Tax Manage...,0


In [244]:
Twitter_data_in_sheets['sent_score'].dtypes
Twitter_data_in_sheets['Cleaned_Tweet'].dtypes

dtype('O')

In [245]:
filtered_BTC_tweets['sent_score'] = filtered_BTC_tweets['sent_score'].astype(float)
twitter_kaggle_training['sent_score'] = twitter_kaggle_training['sent_score'].astype(float)
Twitter_data_in_sheets['sent_score'] = Twitter_data_in_sheets['sent_score'].astype(float)


<ipython-input-245-2e6153e0dbd1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Twitter_data_in_sheets['sent_score'] = Twitter_data_in_sheets['sent_score'].astype(float)


In [246]:
filtered_BTC_tweets.head()

,Cleaned_Tweet,sent_score
0,"Paul Krugman, Nobel Luddite. I had to tweak th...",0.0
1,But (dum b a ss) said you know nothing about ...,0.0
2,Another use case for blockchain and Tipper. Th...,1.0
3,free coins,1.0
4,WE are happy to announce that PayVX Presale Ph...,1.0


In [247]:
filtered_BTC_tweets.shape[0]

50873

In [248]:
twitter_kaggle_training.head()

,Cleaned_Tweet,sent_score
0,I am coming to the borders and I will kill you...,1.0
1,im getting on borderlands and i will kill you ...,1.0
2,im coming on borderlands and i will murder you...,1.0
3,im getting on borderlands 2 and i will murder ...,1.0
4,im getting into borderlands and i can murder y...,1.0


In [249]:
twitter_kaggle_training.shape[0]

61691

In [250]:
Twitter_data_in_sheets.head()

,Cleaned_Tweet,sent_score
0,We are hiring: Senior Software Engineer - Prot...,0.0
1,This is true Amazon Web Services php html htm...,1.0
2,Devops Engineer Aws Ansible Cassandra Mysql Ub...,0.0
3,Happy New Year to all those AWS instances of o...,0.0
4,Amazon is hiring! Sr. International Tax Manage...,0.0


In [251]:
Twitter_data_in_sheets.shape[0]

100001

In [252]:
combined_df = pd.concat([filtered_BTC_tweets, twitter_kaggle_training, Twitter_data_in_sheets], ignore_index=True)

In [253]:
combined_df.shape[0]

212565

In [254]:
combined_df.to_csv(save_df_dir, index=False)

In [256]:
Twitter_data_in_sheets.to_csv(datasets_folder+'/Cleaned_Twitter_data_in_sheets.csv', index =False)

In [257]:
twitter_kaggle_training.to_csv(datasets_folder+'/Cleaned_twitter_kaggle_training.csv', index =False)

In [259]:
filtered_BTC_tweets.to_csv(datasets_folder+'/Cleaned_filtered_BTC_tweets.csv', index =False)